In [28]:
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import torch
import torch.nn as nn
import tensorflow as tf

In [14]:
data = pd.read_csv('./TOA结果/有干扰下TOA.csv')
data = data[['position1','position2','position3','position4','code','X','Y','Z','pre_X','pre_Y','pre_Z']]
data.dropna(axis=0,inplace = True)
data.head()

,position1,position2,position3,position4,code,X,Y,Z,pre_X,pre_Y,pre_Z
0,1270.0,4540.0,4560.0,6310.0,1.0,50.0,50.0,88.0,58.606937,56.774674,67.063874
1,770.0,5000.0,4540.0,6290.0,1.0,50.0,50.0,88.0,35.619764,79.013686,26.564664
2,1240.0,4540.0,4550.0,6300.0,1.0,50.0,50.0,88.0,58.548821,57.634440,69.070773
3,780.0,5010.0,4550.0,6300.0,1.0,50.0,50.0,88.0,34.800407,78.549250,25.507421
4,790.0,5030.0,4550.0,6290.0,1.0,50.0,50.0,88.0,34.661567,80.397355,21.979818


In [22]:
def transform(data):
    whole_tag = []
    name = data.columns[-1]
    for i in data.groupby(name)['Tag0']:
        a = list(i[1])
        if len(a)<4:
            while True:
                a.append('nan')
                if len(a) == 4:
                    break
        time = i[0]
        a.append(time)
        whole_tag.append(a)
    data_fag = pd.DataFrame(whole_tag)
#     print(data_fag.head())
    if len(data_fag.columns) !=5:
        data_fag.drop(list(data_fag.columns[5:]), axis=1 , inplace = True)
    data_fag.columns = ['postion1' ,'postion2','postion3','postion4','time_tag' ]
    data_fag = data_fag.astype('float')
#     data_fag.columns = []
    return data_fag

In [50]:
path  = '.\\附件\\附件数据\\'
df = pd.read_csv(path+'附件4.txt',sep = ':')
df.head()

,T,110936566,DecaRangeRTLS,LogFile,B?m [ ???,Conf,Tag0,1,Chan2
0,T,55943544,RR,0,0,2940,2940,52,19764
1,T,55943544,RR,0,1,4290,4290,52,19764
2,T,55943545,RR,0,2,2840,2840,52,19764
3,T,55943545,RR,0,3,4190,4190,52,19764
4,T,60341974,RR,0,0,5240,5240,171,20907


In [51]:
df= transform(df)
df.drop('time_tag',axis=1,inplace = True)
df
# df.drop('是否干扰',axis=1,inplace=True)
# df

,postion1,postion2,postion3,postion4
0,5050.0,3740.0,3710.0,2070.0
1,5010.0,5320.0,1990.0,2930.0
2,2840.0,4490.0,2860.0,4190.0
3,4840.0,2600.0,4960.0,2700.0
4,5050.0,4110.0,3710.0,2110.0
5,2740.0,2720.0,4670.0,4790.0
6,2940.0,4290.0,2840.0,4190.0
7,5240.0,5360.0,2040.0,2940.0
8,5010.0,4120.0,3810.0,2020.0
9,4800.0,2610.0,4750.0,2550.0


In [52]:
scl_data = pd.read_csv('.\\任务五核弹爆炸好模型\\就是你了.csv')[['position1' ,'position2','position3' ,'position4']]
from sklearn.preprocessing import StandardScaler , RobustScaler
rob = RobustScaler()
rob.fit(scl_data)
df = pd.DataFrame(rob.transform(df))
station_p = np.array([0,0,1300,5000,0,1700,0,5000,1700,5000,5000,1300])/1000
df[['A01','A02','A03','A11','A12','A13','A21','A22','A23','A31','A32','A33']]=station_p
df.head()

,0,1,2,3,A01,A02,A03,A11,A12,A13,A21,A22,A23,A31,A32,A33
0,0.589189,-0.135135,-0.151351,-1.037838,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3
1,0.567568,0.718919,-1.081081,-0.572973,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3
2,-0.605405,0.270270,-0.610811,0.108108,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3
3,0.475676,-0.751351,0.524324,-0.697297,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3
4,0.589189,0.064865,-0.151351,-1.016216,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3


In [53]:

classification = tf.keras.models.load_model('.\\任务五核弹爆炸好模型\\第2次分类结果.h5')
classes = classification.predict(df.values)
classes = np.argmax(classes,axis=1)
# print(pd.DataFrame(classes_no))
# print(pd.DataFrame(classes_yes))
df['是否干扰'] = classes
df

,0,1,2,3,A01,A02,A03,A11,A12,A13,A21,A22,A23,A31,A32,A33,是否干扰
0,0.589189,-0.135135,-0.151351,-1.037838,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3,1
1,0.567568,0.718919,-1.081081,-0.572973,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3,0
2,-0.605405,0.270270,-0.610811,0.108108,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3,0
3,0.475676,-0.751351,0.524324,-0.697297,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3,1
4,0.589189,0.064865,-0.151351,-1.016216,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3,1
5,-0.659459,-0.686486,0.367568,0.432432,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3,0
6,-0.551351,0.162162,-0.621622,0.108108,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3,0
7,0.691892,0.740541,-1.054054,-0.567568,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3,1
8,0.567568,0.070270,-0.097297,-1.064865,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3,1
9,0.454054,-0.745946,0.410811,-0.778378,0.0,0.0,1.3,5.0,0.0,1.7,0.0,5.0,1.7,5.0,5.0,1.3,1


In [54]:

path  = '.\\附件\\附件数据\\'
df = pd.read_csv(path+'附件4.txt',sep = ':')
df= transform(df)
df.drop('time_tag',axis=1,inplace = True)
df['是否干扰'] = classes
df.to_csv('附件四结果.csv')
df

,postion1,postion2,postion3,postion4,是否干扰
0,5050.0,3740.0,3710.0,2070.0,1
1,5010.0,5320.0,1990.0,2930.0,0
2,2840.0,4490.0,2860.0,4190.0,0
3,4840.0,2600.0,4960.0,2700.0,1
4,5050.0,4110.0,3710.0,2110.0,1
5,2740.0,2720.0,4670.0,4790.0,0
6,2940.0,4290.0,2840.0,4190.0,0
7,5240.0,5360.0,2040.0,2940.0,1
8,5010.0,4120.0,3810.0,2020.0,1
9,4800.0,2610.0,4750.0,2550.0,1


In [69]:
test_all = []
for i in df_yes_p.values: #real_data
#     print(i[:4])
    test = i[0:4]
#     print(test)
#     break
    def solve_function(unsolved_value ):
        x,y,z=unsolved_value[0],unsolved_value[1],unsolved_value[2]
#         d1 = np.sqrt(x**2+y**2+(z-1300)**2)
#         d2 = np.sqrt((x-5000)**2+y**2+(z-1700)**2)
#         d3 = np.sqrt((x)**2+(y-5000)**2+(z-1700)**2)
#         d4 = np.sqrt((x-5000)**2+(y-5000)**2+(z-1300)**2)
        return [
            x**2+y**2+(z-1200)**2-test[0]**2, # d1
            (x-5000)**2+y**2+(z-1600)**2-test[1]**2, # d2
             (x)**2+(y-3000)**2+(z-1600)**2-test[2]**2, # d3
            (x-5000)**2+(y-3000)**2+(z-1200)**2-test[3]**2, #d4
            #双曲率
#             test[0]-test[1] - (d1-d2),
#             test[1]-test[2] - (d2-d3),
#             test[2]-test[3] - (d3-d4),
#             test[3]-test[0] - (d4-d1),
            

        ]
    solved=fsolve(solve_function,[0, 0, 0, 0])
    test_all.append(list(solved)[:3])

C:\anaconda\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [65]:
def merge(data,result):
    all_data = []
    result =list( np.array(result)/10)
    for i in data.values:
        all_data.append(list(i))
    step = 0
    for i in all_data:
        i.extend(result[step])
        step+=1
    df = pd.DataFrame(all_data)
    return df

In [70]:
n = merge(df_yes_p,test_all)
n.columns = ['position1','position2','position3','position4','code','X','Y','Z']
n.to_csv('有干扰下.csv')

In [68]:
n

,position1,position2,position3,position4,code,X,Y,Z
0,3550.0,2510.0,3410.0,2140.0,153216041.0,315.449906,171.279003,128.662427
1,4500.0,1940.0,4420.0,1460.0,153251568.0,418.406793,169.214630,110.885684
2,4220.0,2580.0,3730.0,1450.0,153719679.0,363.213070,217.753086,127.313666
3,720.0,4520.0,3050.0,5380.0,202007174.0,51.568406,4.502301,99.586760
4,3300.0,3130.0,2900.0,2790.0,202543707.0,258.629268,187.859761,209.280229


In [22]:
result = pd.DataFrame(all_data)
result.columns = ['position1','position2','position3','position4','code','X','Y','Z','pre_X','pre_Y','pre_Z']
result[['pre_X','pre_Y','pre_Z']] = result[['pre_X','pre_Y','pre_Z']]/10
result

,position1,position2,position3,position4,code,X,Y,Z,pre_X,pre_Y,pre_Z
0,1270.0,4540.0,4560.0,6310.0,1.0,50.0,50.0,88.0,58.606937,56.774674,67.063874
1,770.0,5000.0,4540.0,6290.0,1.0,50.0,50.0,88.0,35.619764,79.013686,26.564664
2,1240.0,4540.0,4550.0,6300.0,1.0,50.0,50.0,88.0,58.548821,57.634440,69.070773
3,780.0,5010.0,4550.0,6300.0,1.0,50.0,50.0,88.0,34.800407,78.549250,25.507421
4,790.0,5030.0,4550.0,6290.0,1.0,50.0,50.0,88.0,34.661567,80.397355,21.979818
...,...,...,...,...,...,...,...,...,...,...,...
10689,1380.0,4080.0,4530.0,6000.0,99.0,100.0,50.0,130.0,98.718997,59.928979,205.879576
10690,1440.0,4080.0,4530.0,6000.0,99.0,100.0,50.0,130.0,99.596596,60.814548,214.591885
10691,1380.0,4090.0,4530.0,6000.0,99.0,100.0,50.0,130.0,98.207393,60.209651,206.531555
10692,1380.0,4100.0,4520.0,6000.0,99.0,100.0,50.0,130.0,97.346052,61.076011,206.896116


In [15]:
final_mean =  data.groupby('code').mean()

In [49]:
# df = pd.read_csv('正常处理后数据普通求解.csv')
# df

,Unnamed: 0,num,position1,position2,position3,position4,code,X,Y,Z,pre_X,pre_Y,pre_Z
0,0,0.0,770.0,4540.0,4540.0,6310.0,1.0,50.0,50.0,88.0,54.056105,54.056105,105.253233
1,1,1.0,770.0,4540.0,4570.0,6290.0,1.0,50.0,50.0,88.0,57.022113,54.295596,90.662446
2,2,2.0,760.0,4550.0,4560.0,6300.0,1.0,50.0,50.0,88.0,55.351762,54.442899,93.697272
3,3,3.0,760.0,4550.0,4560.0,6290.0,1.0,50.0,50.0,88.0,56.065282,55.157120,91.093502
4,4,4.0,750.0,4550.0,4550.0,6310.0,1.0,50.0,50.0,88.0,54.082430,54.082430,99.850155
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6424,6424,6424.0,930.0,4110.0,4520.0,5990.0,99.0,100.0,50.0,130.0,94.131418,59.089972,135.434699
6425,6425,6425.0,930.0,4100.0,4530.0,5980.0,99.0,100.0,50.0,130.0,95.568918,58.815519,132.445769
6426,6426,6426.0,920.0,4090.0,4530.0,6020.0,99.0,100.0,50.0,130.0,93.828396,56.340717,146.432841
6427,6427,6427.0,940.0,4090.0,4510.0,6000.0,99.0,100.0,50.0,130.0,94.282429,58.576171,146.325881


In [32]:
# from sklearn.metrics import mean_squared_error
# mean_squared_error(final_median[['X','Y','Z']],final_median[['pre_X','pre_Y','pre_Z']])

2428.74011241028

In [37]:
# final_median.to_csv('正常数据平均值.csv')

In [57]:
import torch
loss = nn.L1Loss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(torch.from_numpy(final_median[['X','Y','Z']].values),
              torch.from_numpy(final_median[['pre_X','pre_Y','pre_Z']].values))


In [12]:
def precision_describe(data):
    loss = nn.L1Loss()
    real = ['X','Y','Z']
    predict = ['pre_X','pre_Y','pre_Z']
#     xyz = data[['X','Y','Z']]
#     predxyz = data[['pre_X','pre_Y','pre_Z']]
    b = loss(torch.from_numpy(data[['X','Y','Z']].values),
              torch.from_numpy(data[['pre_X','pre_Y','pre_Z']].values))
    print('''['X','Y','Z']精度为''',b)
    # single precision
    for i in range(len(real)):
        a = loss(torch.from_numpy(data[real[i]].values),
              torch.from_numpy(data[predict[i]].values))
        print(real[i],'精度为',a)
    real1 = ['X','Y']
    predict1 = ['pre_X','pre_Y']
    a1 = loss(torch.from_numpy(data[real1].values),
              torch.from_numpy(data[predict1].values))
    real2 = ['X','Z']
    predict2 = ['pre_X','pre_Z']
    a2 = loss(torch.from_numpy(data[real2].values),
              torch.from_numpy(data[predict2].values))
    real3 = ['Y','Z']
    predict3 = ['pre_Y','pre_Z']
    a3 = loss(torch.from_numpy(data[real3].values),
              torch.from_numpy(data[predict3].values))
    print(real1,'精度为',a1)
#     print(real2,'精度为',a2)
#     print(real3,'精度为',a3)
    
    
    

In [16]:
print('有干扰下')
precision_describe(final_mean)

有干扰下
['X','Y','Z']精度为 tensor(24.5935, dtype=torch.float64)
X 精度为 tensor(10.1235, dtype=torch.float64)
Y 精度为 tensor(10.1388, dtype=torch.float64)
Z 精度为 tensor(53.5182, dtype=torch.float64)
['X', 'Y'] 精度为 tensor(10.1311, dtype=torch.float64)


In [13]:
print('无干扰下')
precision_describe(final_mean)

无干扰下
['X','Y','Z']精度为 tensor(7.4005, dtype=torch.float64)
X 精度为 tensor(4.0668, dtype=torch.float64)
Y 精度为 tensor(3.4649, dtype=torch.float64)
Z 精度为 tensor(14.6698, dtype=torch.float64)
['X', 'Y'] 精度为 tensor(3.7659, dtype=torch.float64)
